In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 통합데이터셋 생성

### 현재
- 아동 인구수(완료)
- 아동 인구밀도(완료)

### 미래
- 신혼부부수(2020_2%_인구사항_20240623_06781.csv)
- 출생율(2020_2%_인구사항_20240623_06781.csv)

### 공급 관련 변수
- 달빛 병원수(완료)
- 인력자원 수(지역별 인력자원)
- 달빛 병원 대체 가능 시설수(심야응급의료기관)

### 경제사회변수
- 지역내 총생산

# 시군구코드별_변수_v2 불러와서 오류 확인

In [ ]:
path = '/content/drive/MyDrive/mulcam_bigdata/2024통계데이터공모전/기획_데이터수집/성원님/temp/시군구코드별_변수_v2.csv'
df = pd.read_csv(path)
df

,중분류,행정구역시도명,행정구역시군구명,아동인구수,병원개수,인구 천명당 병원수,면적,시군구코드,소아청소년수,출생아수,가임연령부부수,2차병원수,3차병원수
0,31050,경기도,부천시,1801,1.0,0.000555,53.45,31050,2227,791,1203,8.0,0.0
1,31240,경기도,화성시,1785,2.0,0.001120,698.18,31240,2319,991,1450,10.0,0.0
2,31023,경기도,분당구,1736,0.0,0.000000,69.49,31023,2093,580,969,5.0,1.0
3,31130,경기도,남양주시,1472,1.0,0.000679,458.14,31130,1816,646,827,5.0,0.0
4,11230,서울특별시,강남구,1405,2.0,0.001423,39.50,11230,1634,369,576,7.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,32410,강원도,양양군,82,0.0,0.000000,630.01,32410,99,42,53,0.0,0.0
246,37330,경상북도,청송군,72,0.0,0.000000,846.12,37330,94,39,46,1.0,0.0
247,37340,경상북도,영양군,67,0.0,0.000000,815.75,37340,78,24,24,0.0,0.0
248,37310,경상북도,군위군,62,0.0,0.000000,614.32,37310,69,21,25,0.0,0.0


In [ ]:
# 중복 컬럼 제거 ['아동인구수', '시군구코드']
df = df.drop(['아동인구수', '시군구코드'], axis=1)
df

,중분류,행정구역시도명,행정구역시군구명,병원개수,인구 천명당 병원수,면적,소아청소년수,출생아수,가임연령부부수,2차병원수,3차병원수
0,31050,경기도,부천시,1.0,0.000555,53.45,2227,791,1203,8.0,0.0
1,31240,경기도,화성시,2.0,0.001120,698.18,2319,991,1450,10.0,0.0
2,31023,경기도,분당구,0.0,0.000000,69.49,2093,580,969,5.0,1.0
3,31130,경기도,남양주시,1.0,0.000679,458.14,1816,646,827,5.0,0.0
4,11230,서울특별시,강남구,2.0,0.001423,39.50,1634,369,576,7.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
245,32410,강원도,양양군,0.0,0.000000,630.01,99,42,53,0.0,0.0
246,37330,경상북도,청송군,0.0,0.000000,846.12,94,39,46,1.0,0.0
247,37340,경상북도,영양군,0.0,0.000000,815.75,78,24,24,0.0,0.0
248,37310,경상북도,군위군,0.0,0.000000,614.32,69,21,25,0.0,0.0


### 컬럼별 오류 마지막 확인
- 오류확인 필요한 컬럼 ['병원개수']
- 나머지는 전처리하는 과정에서 오류 없는 것 확인 후 추가하여 오류 확인 필요 없음

In [ ]:
# 전국 달빛 병원 개수 90
# df 의 병원개수는 69개
# 누락된 병원 개수 21개
df.병원개수.sum()

69.0

# 달빛병원수 (moon_code)
- 원인 파악을 위한 전처리 코드 불러옴

In [ ]:
moon = pd.read_csv('/content/drive/MyDrive/mulcam_bigdata/2024통계데이터공모전/기획_데이터수집/성원님/Data/달빛병원리스트.csv', index_col=0)

moon

,병원명,주소
0,의료법인우리아이들의료재단우리아이들병원,"서울특별시 구로구 새말로 15, 2~9층 (구로동, 삼전솔하임)"
1,24시열린의원,"서울특별시 광진구 능동로 90, 3층 (자양동, 더클래식500)"
2,연세곰돌이소아청소년과의원,"서울특별시 서초구 방배로 226, 넥센강남타워 3층 (방배동)"
3,연세의원,"서울특별시 강서구 화곡로 197, 1,2층 (화곡동)"
4,미즈아이프라자산부인과의원,"서울특별시 노원구 동일로 1669, 수락프라자 (상계동)"
...,...,...
85,서울패밀리병원,"경상남도 창원시 성산구 마디미서로 54, 서울아동병원(2,7,8,9,11,12층) ..."
86,양덕서울아동병원,경상남도 창원시 마산회원구 양덕로 190 (석전동)
87,우리들소아과의원,"제주특별자치도 서귀포시 중정로 70, (서귀동)"
88,탑동365일의원,"제주특별자치도 제주시 탑동로 24 (삼도이동,2층)"


In [ ]:
# 주소 데이터를 시도와 시군구로 구분

moon['시도'] = moon['주소'].apply(lambda x: x.split(' ', 1)[0])
moon['시군구'] = moon['주소'].apply(lambda x: x.split(' ', 2)[1])
moon['시군구2'] = moon['주소'].apply(lambda x: x.split(' ', 3)[2])


# 결과 출력
moon

,병원명,주소,시도,시군구,시군구2
0,의료법인우리아이들의료재단우리아이들병원,"서울특별시 구로구 새말로 15, 2~9층 (구로동, 삼전솔하임)",서울특별시,구로구,새말로
1,24시열린의원,"서울특별시 광진구 능동로 90, 3층 (자양동, 더클래식500)",서울특별시,광진구,능동로
2,연세곰돌이소아청소년과의원,"서울특별시 서초구 방배로 226, 넥센강남타워 3층 (방배동)",서울특별시,서초구,방배로
3,연세의원,"서울특별시 강서구 화곡로 197, 1,2층 (화곡동)",서울특별시,강서구,화곡로
4,미즈아이프라자산부인과의원,"서울특별시 노원구 동일로 1669, 수락프라자 (상계동)",서울특별시,노원구,동일로
...,...,...,...,...,...
85,서울패밀리병원,"경상남도 창원시 성산구 마디미서로 54, 서울아동병원(2,7,8,9,11,12층) ...",경상남도,창원시,성산구
86,양덕서울아동병원,경상남도 창원시 마산회원구 양덕로 190 (석전동),경상남도,창원시,마산회원구
87,우리들소아과의원,"제주특별자치도 서귀포시 중정로 70, (서귀동)",제주특별자치도,서귀포시,중정로
88,탑동365일의원,"제주특별자치도 제주시 탑동로 24 (삼도이동,2층)",제주특별자치도,제주시,탑동로


In [ ]:
# 결측치 확인
moon.isnull().sum()

병원명     0
주소      0
시도      0
시군구     0
시군구2    0
dtype: int64

### 시군구코드 추가를 위한 시도시군구 명 컬럼 생성
- 시도 형식을 맞춰줘야함 >> 형식 매핑을 위한 딕셔너리 정의

In [ ]:
# moon['시도'] 형태는 다음과 같음
moon['시도'].unique()

# 시도시군구에 들어갈 시도 형태는 2글자 형식이어야함

array(['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시',
       '세종특별자치시', '경기도', '강원특별자치도', '충청북도', '충청남도', '전북특별자치도', '전라남도',
       '경상남도', '제주특별자치도'], dtype=object)

In [ ]:
# 형식 매핑을 위한 딕셔너리 정의
map1 = {
    '서울특별시' : '서울',
    '부산광역시' : '부산',
    '대구광역시' : '대구',
    '인천광역시' : '인천',
    '광주광역시' : '광주',
    '대전광역시' : '대전',
    '울산광역시' : '울산',
    '세종특별자치시' : '세종특별자치시',
    '경기도' : '경기',
    '강원특별자치도' : '강원',
    '충청북도' : '충북',
    '충청남도' : '충남',
    '전북특별자치도' : '전북',
    '전라남도' : '전남',
    '경상북도' : '경북',
    '경상남도' : '경남',
    '제주특별자치도' : '제주'
}

In [ ]:
# 매핑
moon['시도2'] = moon['시도'].astype(str).map(map1)

In [ ]:
# 결과물 확인
moon

,병원명,주소,시도,시군구,시군구2,시도2
0,의료법인우리아이들의료재단우리아이들병원,"서울특별시 구로구 새말로 15, 2~9층 (구로동, 삼전솔하임)",서울특별시,구로구,새말로,서울
1,24시열린의원,"서울특별시 광진구 능동로 90, 3층 (자양동, 더클래식500)",서울특별시,광진구,능동로,서울
2,연세곰돌이소아청소년과의원,"서울특별시 서초구 방배로 226, 넥센강남타워 3층 (방배동)",서울특별시,서초구,방배로,서울
3,연세의원,"서울특별시 강서구 화곡로 197, 1,2층 (화곡동)",서울특별시,강서구,화곡로,서울
4,미즈아이프라자산부인과의원,"서울특별시 노원구 동일로 1669, 수락프라자 (상계동)",서울특별시,노원구,동일로,서울
...,...,...,...,...,...,...
85,서울패밀리병원,"경상남도 창원시 성산구 마디미서로 54, 서울아동병원(2,7,8,9,11,12층) ...",경상남도,창원시,성산구,경남
86,양덕서울아동병원,경상남도 창원시 마산회원구 양덕로 190 (석전동),경상남도,창원시,마산회원구,경남
87,우리들소아과의원,"제주특별자치도 서귀포시 중정로 70, (서귀동)",제주특별자치도,서귀포시,중정로,제주
88,탑동365일의원,"제주특별자치도 제주시 탑동로 24 (삼도이동,2층)",제주특별자치도,제주시,탑동로,제주


In [ ]:
# 시도시군구 컬럼 생성 >> 시도시군구 코드를 추가할 때 기준으로 사용하기 위함
moon['시도시군구'] = moon['시도2'] + ' ' + moon['시군구']
moon

,병원명,주소,시도,시군구,시군구2,시도2,시도시군구
0,의료법인우리아이들의료재단우리아이들병원,"서울특별시 구로구 새말로 15, 2~9층 (구로동, 삼전솔하임)",서울특별시,구로구,새말로,서울,서울 구로구
1,24시열린의원,"서울특별시 광진구 능동로 90, 3층 (자양동, 더클래식500)",서울특별시,광진구,능동로,서울,서울 광진구
2,연세곰돌이소아청소년과의원,"서울특별시 서초구 방배로 226, 넥센강남타워 3층 (방배동)",서울특별시,서초구,방배로,서울,서울 서초구
3,연세의원,"서울특별시 강서구 화곡로 197, 1,2층 (화곡동)",서울특별시,강서구,화곡로,서울,서울 강서구
4,미즈아이프라자산부인과의원,"서울특별시 노원구 동일로 1669, 수락프라자 (상계동)",서울특별시,노원구,동일로,서울,서울 노원구
...,...,...,...,...,...,...,...
85,서울패밀리병원,"경상남도 창원시 성산구 마디미서로 54, 서울아동병원(2,7,8,9,11,12층) ...",경상남도,창원시,성산구,경남,경남 창원시
86,양덕서울아동병원,경상남도 창원시 마산회원구 양덕로 190 (석전동),경상남도,창원시,마산회원구,경남,경남 창원시
87,우리들소아과의원,"제주특별자치도 서귀포시 중정로 70, (서귀동)",제주특별자치도,서귀포시,중정로,제주,제주 서귀포시
88,탑동365일의원,"제주특별자치도 제주시 탑동로 24 (삼도이동,2층)",제주특별자치도,제주시,탑동로,제주,제주 제주시


In [ ]:
# 결측치가 없이 잘 매핑됨
moon.isnull().sum()

병원명      0
주소       0
시도       0
시군구      0
시군구2     0
시도2      0
시도시군구    0
dtype: int64

기준 데이터 분류 형태와 다른 시도군구 데이터 매핑해줄 딕셔너리 정의

In [ ]:
map2 = {'경기 수원시': {'경기 장안구': 31011, '경기 권선구': 31012, '경기 팔달구': 31013},
            '경기 성남시': {'경기 중원구': 31022, '경기 분당구': 31023},
            '경기 안양시': {'경기 만안구': 31041, '경기 동안구': 31042},
            '경기 안산시': {'경기 상록구': 31091, '경기 단원구': 31092},
            '경기 고양시': {'경기 일산동구': 31103, '경기 일산서구': 31104},
            '경기 용인시': {'경기 처인구': 31191, '경기 기흥구': 31192, '경기 수지구': 31193},
            '충북 청주시': {'충북 상당구': 33041, '충북 서원구': 33042, '충북 흥덕구': 33043, '충북 청원구': 33044},
            '충남 천안시': {'충남 동남구': 34011, '충남 서북구': 34012},
            '전북 전주시': {'전북 완산구': 35011, '전북 덕진구': 35012},
            '경북 포항시': {'경북 포항시 남구': 37011, '경북 포항시 북구': 37012},
            '경남 창원시': {'경남 창원시 의창구': 38111, '경남 창원시 성산구': 38112, '경남 창원시 마산합포구': 38113, '경남 창원시 마산회원구': 38114, '경남 창원시 진해구': 38115}}

# map2에서도 세분화 방법에 따라 2가지 유형으로 나뉜다
# 세분화 유형1. 시도 + 구이름
# 세분화 유형2. 시도 + 시이름 + 구이름
# 따라서 케이스를 달리해서 처리해야한다

for 문을 만들기 위한 파싱

In [ ]:
list(map2.keys())[0]

'경기 수원시'

In [ ]:
moon[moon['시도시군구'] == list(map2.keys())[0]]

,병원명,주소,시도,시군구,시군구2,시도2,시도시군구
43,아주맑은소아청소년과의원,"경기도 수원시 영통구 효원로 363 (매탄동, 303상가 B202호, 203호)",경기도,수원시,영통구,경기,경기 수원시
44,성모수메디컬의원,"경기도 수원시 권선구 금곡로 112, 이수프라자 304,305,306,308호 (금곡동)",경기도,수원시,권선구,경기,경기 수원시


In [ ]:
temp = moon[moon['시도시군구'] == list(map2.keys())[0]]
temp

,병원명,주소,시도,시군구,시군구2,시도2,시도시군구
43,아주맑은소아청소년과의원,"경기도 수원시 영통구 효원로 363 (매탄동, 303상가 B202호, 203호)",경기도,수원시,영통구,경기,경기 수원시
44,성모수메디컬의원,"경기도 수원시 권선구 금곡로 112, 이수프라자 304,305,306,308호 (금곡동)",경기도,수원시,권선구,경기,경기 수원시


In [ ]:
temp['시도시군구'] = temp['시도2']+ ' ' + temp['시군구2']

<ipython-input-19-56f68a78b035>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['시도시군구'] = temp['시도2']+ ' ' + temp['시군구2']


In [ ]:
temp

,병원명,주소,시도,시군구,시군구2,시도2,시도시군구
43,아주맑은소아청소년과의원,"경기도 수원시 영통구 효원로 363 (매탄동, 303상가 B202호, 203호)",경기도,수원시,영통구,경기,경기 영통구
44,성모수메디컬의원,"경기도 수원시 권선구 금곡로 112, 이수프라자 304,305,306,308호 (금곡동)",경기도,수원시,권선구,경기,경기 권선구


파싱 코드 일반화 >> for문 생성 후 돌리기

In [ ]:
# 데이터 원본 복사
moon_temp = moon.copy()

In [ ]:
moon_temp[moon_temp['시군구'] == '수원시']

,병원명,주소,시도,시군구,시군구2,시도2,시도시군구
43,아주맑은소아청소년과의원,"경기도 수원시 영통구 효원로 363 (매탄동, 303상가 B202호, 203호)",경기도,수원시,영통구,경기,경기 수원시
44,성모수메디컬의원,"경기도 수원시 권선구 금곡로 112, 이수프라자 304,305,306,308호 (금곡동)",경기도,수원시,권선구,경기,경기 수원시


In [ ]:
# # 세분화 방법에 따른 유형 2가지 고려안한 for 문
# for i in range(len(map2)):
#     temp_df = moon_temp[moon_temp['시도시군구'] == list(map2.keys())[i]]
#     temp_df['시도시군구'] = temp_df['시도2']+ ' ' + temp_df['시군구2']
#     moon_temp[moon_temp['시도시군구'] == list(map2.keys())[i]] = temp_df

In [ ]:
# 세분화 방법에 따른 유형 2가지 고려한 for 문
for i in range(len(map2)):

    # 세분화 유형 2에 해당하는 케이스
    if list(map2.keys())[i] in ['경북 포항시', '경남 창원시']:
        temp_df = moon_temp[moon_temp['시도시군구'] == list(map2.keys())[i]]
        temp_df['시도시군구'] = temp_df['시도2']+ ' ' + temp_df['시군구'] + ' ' + temp_df['시군구2']
        moon_temp[moon_temp['시도시군구'] == list(map2.keys())[i]] = temp_df

    # 세분화 유형 1에 해당하는 케이스
    else:
        temp_df = moon_temp[moon_temp['시도시군구'] == list(map2.keys())[i]]
        temp_df['시도시군구'] = temp_df['시도2']+ ' ' + temp_df['시군구2']
        moon_temp[moon_temp['시도시군구'] == list(map2.keys())[i]] = temp_df

<ipython-input-24-971ff6794872>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['시도시군구'] = temp_df['시도2']+ ' ' + temp_df['시군구2']
<ipython-input-24-971ff6794872>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['시도시군구'] = temp_df['시도2']+ ' ' + temp_df['시군구'] + ' ' + temp_df['시군구2']


결과 확인

In [ ]:
map2

{'경기 수원시': {'경기 장안구': 31011, '경기 권선구': 31012, '경기 팔달구': 31013},
 '경기 성남시': {'경기 중원구': 31022, '경기 분당구': 31023},
 '경기 안양시': {'경기 만안구': 31041, '경기 동안구': 31042},
 '경기 안산시': {'경기 상록구': 31091, '경기 단원구': 31092},
 '경기 고양시': {'경기 일산동구': 31103, '경기 일산서구': 31104},
 '경기 용인시': {'경기 처인구': 31191, '경기 기흥구': 31192, '경기 수지구': 31193},
 '충북 청주시': {'충북 상당구': 33041,
  '충북 서원구': 33042,
  '충북 흥덕구': 33043,
  '충북 청원구': 33044},
 '충남 천안시': {'충남 동남구': 34011, '충남 서북구': 34012},
 '전북 전주시': {'전북 완산구': 35011, '전북 덕진구': 35012},
 '경북 포항시': {'경북 포항시 남구': 37011, '경북 포항시 북구': 37012},
 '경남 창원시': {'경남 창원시 의창구': 38111,
  '경남 창원시 성산구': 38112,
  '경남 창원시 마산합포구': 38113,
  '경남 창원시 마산회원구': 38114,
  '경남 창원시 진해구': 38115}}

세분화 1유형 결과

In [ ]:
moon_temp[moon_temp['시군구'] == '수원시']

,병원명,주소,시도,시군구,시군구2,시도2,시도시군구
43,아주맑은소아청소년과의원,"경기도 수원시 영통구 효원로 363 (매탄동, 303상가 B202호, 203호)",경기도,수원시,영통구,경기,경기 영통구
44,성모수메디컬의원,"경기도 수원시 권선구 금곡로 112, 이수프라자 304,305,306,308호 (금곡동)",경기도,수원시,권선구,경기,경기 권선구


In [ ]:
moon_temp[moon_temp['시군구'] == '고양시']

,병원명,주소,시도,시군구,시군구2,시도2,시도시군구
45,일산우리들소아청소년과의원,"경기도 고양시 일산서구 중앙로 1426, 일송노블레스 207호, 208(일부)호, ...",경기도,고양시,일산서구,경기,경기 일산서구
60,연세열린의원,"경기도 고양시 덕양구 지축1로 39, 401호 (지축동)",경기도,고양시,덕양구,경기,경기 덕양구


세분화 2 유형 결과

In [ ]:
moon_temp[moon_temp['시군구'] == '창원시']

,병원명,주소,시도,시군구,시군구2,시도2,시도시군구
85,서울패밀리병원,"경상남도 창원시 성산구 마디미서로 54, 서울아동병원(2,7,8,9,11,12층) ...",경상남도,창원시,성산구,경남,경남 창원시 성산구
86,양덕서울아동병원,경상남도 창원시 마산회원구 양덕로 190 (석전동),경상남도,창원시,마산회원구,경남,경남 창원시 마산회원구


행정구역분류코드와 병합

In [ ]:
path = '/content/drive/MyDrive/mulcam_bigdata/2024통계데이터공모전/기획_데이터수집/성원님/temp/한국행정구역분류_행정동코드(7자리)_20210701기준.xlsx'
dong = pd.read_excel(path, sheet_name='1. 총괄표(현행)', header=2, index_col=None)

In [ ]:
dong = dong.drop(columns=[dong.columns[0]])
dong.fillna(0, inplace=True)

In [ ]:
dong

,대분류,시도,중분류,시군구,소분류,읍면동,영문 표기,한자 표기
0,11.0,서울특별시,0.0,0,0.0,0,Seoul,서울特別市
1,11.0,서울특별시,11010.0,종로구,0.0,0,Jongno-gu,鐘路區
2,11.0,서울특별시,11010.0,종로구,1101053.0,사직동,Sajik-dong,社稷洞
3,11.0,서울특별시,11010.0,종로구,1101054.0,삼청동,Samcheong-dong,三淸洞
4,11.0,서울특별시,11010.0,종로구,1101055.0,부암동,Buam-dong,付岩洞
...,...,...,...,...,...,...,...,...
3787,39.0,제주특별자치도,39020.0,서귀포시,3902058.0,서홍동,Seohong-dong,西烘洞
3788,39.0,제주특별자치도,39020.0,서귀포시,3902059.0,대륜동,Daeryun-dong,大倫洞
3789,39.0,제주특별자치도,39020.0,서귀포시,3902060.0,대천동,Daecheon-dong,大川洞
3790,39.0,제주특별자치도,39020.0,서귀포시,3902061.0,중문동,Jungmun-dong,中文洞


In [ ]:
# '중분류' 열을 문자열로 변환하면서 소수점 제거
dong['중분류'] = dong['중분류'].apply(lambda x: str(int(x)) if pd.notnull(x) else '')

# 결과 출력
dong

,대분류,시도,중분류,시군구,소분류,읍면동,영문 표기,한자 표기
0,11.0,서울특별시,0,0,0.0,0,Seoul,서울特別市
1,11.0,서울특별시,11010,종로구,0.0,0,Jongno-gu,鐘路區
2,11.0,서울특별시,11010,종로구,1101053.0,사직동,Sajik-dong,社稷洞
3,11.0,서울특별시,11010,종로구,1101054.0,삼청동,Samcheong-dong,三淸洞
4,11.0,서울특별시,11010,종로구,1101055.0,부암동,Buam-dong,付岩洞
...,...,...,...,...,...,...,...,...
3787,39.0,제주특별자치도,39020,서귀포시,3902058.0,서홍동,Seohong-dong,西烘洞
3788,39.0,제주특별자치도,39020,서귀포시,3902059.0,대륜동,Daeryun-dong,大倫洞
3789,39.0,제주특별자치도,39020,서귀포시,3902060.0,대천동,Daecheon-dong,大川洞
3790,39.0,제주특별자치도,39020,서귀포시,3902061.0,중문동,Jungmun-dong,中文洞


In [ ]:
# 데이터 조작을 위한 원본 살리기
dong_temp = dong.copy()
dong_temp

,대분류,시도,중분류,시군구,소분류,읍면동,영문 표기,한자 표기
0,11.0,서울특별시,0,0,0.0,0,Seoul,서울特別市
1,11.0,서울특별시,11010,종로구,0.0,0,Jongno-gu,鐘路區
2,11.0,서울특별시,11010,종로구,1101053.0,사직동,Sajik-dong,社稷洞
3,11.0,서울특별시,11010,종로구,1101054.0,삼청동,Samcheong-dong,三淸洞
4,11.0,서울특별시,11010,종로구,1101055.0,부암동,Buam-dong,付岩洞
...,...,...,...,...,...,...,...,...
3787,39.0,제주특별자치도,39020,서귀포시,3902058.0,서홍동,Seohong-dong,西烘洞
3788,39.0,제주특별자치도,39020,서귀포시,3902059.0,대륜동,Daeryun-dong,大倫洞
3789,39.0,제주특별자치도,39020,서귀포시,3902060.0,대천동,Daecheon-dong,大川洞
3790,39.0,제주특별자치도,39020,서귀포시,3902061.0,중문동,Jungmun-dong,中文洞


In [ ]:
# 데이터 조작에 불필요한 열 제거
dong_temp = dong_temp.drop(['대분류','소분류','영문 표기','한자 표기'], axis=1)
dong_temp

,시도,중분류,시군구,읍면동
0,서울특별시,0,0,0
1,서울특별시,11010,종로구,0
2,서울특별시,11010,종로구,사직동
3,서울특별시,11010,종로구,삼청동
4,서울특별시,11010,종로구,부암동
...,...,...,...,...
3787,제주특별자치도,39020,서귀포시,서홍동
3788,제주특별자치도,39020,서귀포시,대륜동
3789,제주특별자치도,39020,서귀포시,대천동
3790,제주특별자치도,39020,서귀포시,중문동


In [ ]:
dong['시도'].unique()

array(['서울특별시', 0, '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시',
       '세종특별자치시', '경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도',
       '경상남도', '제주특별자치도'], dtype=object)

In [ ]:
# 형식 매핑을 위한 딕셔너리 정의
map3 = {
    '서울특별시' : '서울',
    '부산광역시' : '부산',
    '대구광역시' : '대구',
    '인천광역시' : '인천',
    '광주광역시' : '광주',
    '대전광역시' : '대전',
    '울산광역시' : '울산',
    '세종특별자치시' : '세종특별자치시',
    '경기도' : '경기',
    '강원도' : '강원',
    '충청북도' : '충북',
    '충청남도' : '충남',
    '전라북도' : '전북',
    '전라남도' : '전남',
    '경상북도' : '경북',
    '경상남도' : '경남',
    '제주특별자치도' : '제주'
}

In [ ]:
# 매핑
dong_temp['시도2'] = dong_temp['시도'].astype(str).map(map3)
dong_temp

,시도,중분류,시군구,읍면동,시도2
0,서울특별시,0,0,0,서울
1,서울특별시,11010,종로구,0,서울
2,서울특별시,11010,종로구,사직동,서울
3,서울특별시,11010,종로구,삼청동,서울
4,서울특별시,11010,종로구,부암동,서울
...,...,...,...,...,...
3787,제주특별자치도,39020,서귀포시,서홍동,제주
3788,제주특별자치도,39020,서귀포시,대륜동,제주
3789,제주특별자치도,39020,서귀포시,대천동,제주
3790,제주특별자치도,39020,서귀포시,중문동,제주


결측치 및 이상치 처리

In [ ]:
# 시도2 열의 결측치 처리
dong_temp['시도2'].unique()

array(['서울', nan, '부산', '대구', '인천', '광주', '대전', '울산', '세종특별자치시', '경기',
       '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주'], dtype=object)

In [ ]:
# 결측치 확인 후 처리
dong_temp[dong_temp['시도2'].isnull()]

,시도,중분류,시군구,읍면동,시도2
452,0,0,0,0,NaN
675,0,0,0,0,NaN
826,0,0,0,0,NaN
993,0,0,0,0,NaN
1096,0,0,0,0,NaN
1182,0,0,0,0,NaN
1245,0,0,0,0,NaN
1268,0,0,0,0,NaN
1867,0,0,0,0,NaN
2080,0,0,0,0,NaN


In [ ]:
# 시도2 열의 결측치가진 행 삭제 dropna
dong_temp = dong_temp.dropna(subset=['시도2'])

In [ ]:
# 결측치 삭제 결과 확인
dong_temp['시도2'].isnull().sum()

0

In [ ]:
# 시군구 열 데이터 값이 0인 행 삭제하기 >> 문자열 컬럼에 숫자형이 끼어 있음
dong_temp['시군구'].unique()

array([0, '종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구',
       '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구',
       '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구', '서구', '동구',
       '영도구', '부산진구', '동래구', '남구', '북구', '해운대구', '사하구', '금정구', '연제구',
       '수영구', '사상구', '기장군', '수성구', '달서구', '달성군', '연수구', '남동구', '부평구',
       '계양구', '미추홀구', '강화군', '옹진군', '광산구', '유성구', '대덕구', '울주군', '세종특별자치시',
       '수원시', '장안구', '권선구', '팔달구', '영통구', '성남시', '수정구', '중원구', '분당구',
       '의정부시', '안양시', '만안구', '동안구', '부천시', '광명시', '평택시', '동두천시', '안산시',
       '상록구', '단원구', '고양시', '덕양구', '일산동구', '일산서구', '과천시', '구리시', '남양주시',
       '오산시', '시흥시', '군포시', '의왕시', '하남시', '용인시', '처인구', '기흥구', '수지구',
       '파주시', '이천시', '안성시', '김포시', '화성시', '광주시', '양주시', '포천시', '여주시',
       '연천군', '가평군', '양평군', '춘천시', '원주시', '강릉시', '동해시', '태백시', '속초시',
       '삼척시', '홍천군', '횡성군', '영월군', '평창군', '정선군', '철원군', '화천군', '양구군',
       '인제군', '고성군', '양양군', '충주시', '제천시', '청주시', '상당구', '서원구', '흥덕구',
       

In [ ]:
# 0인 행 제외한 df로 업데이트
dong_temp = dong_temp[dong_temp['시군구'] != 0]

In [ ]:
dong_temp['시군구'].unique()

array(['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구',
       '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구',
       '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구', '서구', '동구',
       '영도구', '부산진구', '동래구', '남구', '북구', '해운대구', '사하구', '금정구', '연제구',
       '수영구', '사상구', '기장군', '수성구', '달서구', '달성군', '연수구', '남동구', '부평구',
       '계양구', '미추홀구', '강화군', '옹진군', '광산구', '유성구', '대덕구', '울주군', '세종특별자치시',
       '수원시', '장안구', '권선구', '팔달구', '영통구', '성남시', '수정구', '중원구', '분당구',
       '의정부시', '안양시', '만안구', '동안구', '부천시', '광명시', '평택시', '동두천시', '안산시',
       '상록구', '단원구', '고양시', '덕양구', '일산동구', '일산서구', '과천시', '구리시', '남양주시',
       '오산시', '시흥시', '군포시', '의왕시', '하남시', '용인시', '처인구', '기흥구', '수지구',
       '파주시', '이천시', '안성시', '김포시', '화성시', '광주시', '양주시', '포천시', '여주시',
       '연천군', '가평군', '양평군', '춘천시', '원주시', '강릉시', '동해시', '태백시', '속초시',
       '삼척시', '홍천군', '횡성군', '영월군', '평창군', '정선군', '철원군', '화천군', '양구군',
       '인제군', '고성군', '양양군', '충주시', '제천시', '청주시', '상당구', '서원구', '흥덕구',
       '청원

In [ ]:
dong_temp['시도시군구'] = dong_temp['시도2'] + ' ' + dong_temp['시군구']
dong_temp

,시도,중분류,시군구,읍면동,시도2,시도시군구
1,서울특별시,11010,종로구,0,서울,서울 종로구
2,서울특별시,11010,종로구,사직동,서울,서울 종로구
3,서울특별시,11010,종로구,삼청동,서울,서울 종로구
4,서울특별시,11010,종로구,부암동,서울,서울 종로구
5,서울특별시,11010,종로구,평창동,서울,서울 종로구
...,...,...,...,...,...,...
3787,제주특별자치도,39020,서귀포시,서홍동,제주,제주 서귀포시
3788,제주특별자치도,39020,서귀포시,대륜동,제주,제주 서귀포시
3789,제주특별자치도,39020,서귀포시,대천동,제주,제주 서귀포시
3790,제주특별자치도,39020,서귀포시,중문동,제주,제주 서귀포시


In [ ]:
dong_temp_res = pd.DataFrame(dong_temp.groupby(['중분류', '시도시군구'])['읍면동'].count())
dong_temp_res

,,읍면동
중분류,시도시군구,
11010,서울 종로구,18
11020,서울 중구,16
11030,서울 용산구,17
11040,서울 성동구,18
11050,서울 광진구,16
...,...,...
38380,경남 함양군,12
38390,경남 거창군,13
38400,경남 합천군,18


In [ ]:
dong_temp_res = dong_temp_res.reset_index()
dong_temp_res = dong_temp_res.drop('읍면동', axis=1)
dong_temp_res

,중분류,시도시군구
0,11010,서울 종로구
1,11020,서울 중구
2,11030,서울 용산구
3,11040,서울 성동구
4,11050,서울 광진구
...,...,...
256,38380,경남 함양군
257,38390,경남 거창군
258,38400,경남 합천군
259,39010,제주 제주시


# 달빛병원 결과, 행정구역분류 결과 병합하기

In [ ]:
# 행정 구역 분류 결과
dong_temp_res

,중분류,시도시군구
0,11010,서울 종로구
1,11020,서울 중구
2,11030,서울 용산구
3,11040,서울 성동구
4,11050,서울 광진구
...,...,...
256,38380,경남 함양군
257,38390,경남 거창군
258,38400,경남 합천군
259,39010,제주 제주시


In [ ]:
dong_temp_res[dong_temp_res['중분류'] == '31010']

,중분류,시도시군구
75,31010,경기 수원시


In [ ]:
# 달빛병원 결과
moon_temp

,병원명,주소,시도,시군구,시군구2,시도2,시도시군구
0,의료법인우리아이들의료재단우리아이들병원,"서울특별시 구로구 새말로 15, 2~9층 (구로동, 삼전솔하임)",서울특별시,구로구,새말로,서울,서울 구로구
1,24시열린의원,"서울특별시 광진구 능동로 90, 3층 (자양동, 더클래식500)",서울특별시,광진구,능동로,서울,서울 광진구
2,연세곰돌이소아청소년과의원,"서울특별시 서초구 방배로 226, 넥센강남타워 3층 (방배동)",서울특별시,서초구,방배로,서울,서울 서초구
3,연세의원,"서울특별시 강서구 화곡로 197, 1,2층 (화곡동)",서울특별시,강서구,화곡로,서울,서울 강서구
4,미즈아이프라자산부인과의원,"서울특별시 노원구 동일로 1669, 수락프라자 (상계동)",서울특별시,노원구,동일로,서울,서울 노원구
...,...,...,...,...,...,...,...
85,서울패밀리병원,"경상남도 창원시 성산구 마디미서로 54, 서울아동병원(2,7,8,9,11,12층) ...",경상남도,창원시,성산구,경남,경남 창원시 성산구
86,양덕서울아동병원,경상남도 창원시 마산회원구 양덕로 190 (석전동),경상남도,창원시,마산회원구,경남,경남 창원시 마산회원구
87,우리들소아과의원,"제주특별자치도 서귀포시 중정로 70, (서귀동)",제주특별자치도,서귀포시,중정로,제주,제주 서귀포시
88,탑동365일의원,"제주특별자치도 제주시 탑동로 24 (삼도이동,2층)",제주특별자치도,제주시,탑동로,제주,제주 제주시


In [ ]:
temp_result = moon_temp.merge(dong_temp_res, on='시도시군구', how='left', suffixes=('_moon','_dong'))
temp_result

,병원명,주소,시도,시군구,시군구2,시도2,시도시군구,중분류
0,의료법인우리아이들의료재단우리아이들병원,"서울특별시 구로구 새말로 15, 2~9층 (구로동, 삼전솔하임)",서울특별시,구로구,새말로,서울,서울 구로구,11170
1,24시열린의원,"서울특별시 광진구 능동로 90, 3층 (자양동, 더클래식500)",서울특별시,광진구,능동로,서울,서울 광진구,11050
2,연세곰돌이소아청소년과의원,"서울특별시 서초구 방배로 226, 넥센강남타워 3층 (방배동)",서울특별시,서초구,방배로,서울,서울 서초구,11220
3,연세의원,"서울특별시 강서구 화곡로 197, 1,2층 (화곡동)",서울특별시,강서구,화곡로,서울,서울 강서구,11160
4,미즈아이프라자산부인과의원,"서울특별시 노원구 동일로 1669, 수락프라자 (상계동)",서울특별시,노원구,동일로,서울,서울 노원구,11110
...,...,...,...,...,...,...,...,...
85,서울패밀리병원,"경상남도 창원시 성산구 마디미서로 54, 서울아동병원(2,7,8,9,11,12층) ...",경상남도,창원시,성산구,경남,경남 창원시 성산구,NaN
86,양덕서울아동병원,경상남도 창원시 마산회원구 양덕로 190 (석전동),경상남도,창원시,마산회원구,경남,경남 창원시 마산회원구,NaN
87,우리들소아과의원,"제주특별자치도 서귀포시 중정로 70, (서귀동)",제주특별자치도,서귀포시,중정로,제주,제주 서귀포시,39020
88,탑동365일의원,"제주특별자치도 제주시 탑동로 24 (삼도이동,2층)",제주특별자치도,제주시,탑동로,제주,제주 제주시,39010


In [ ]:
# 결측치 확인
temp_result.isnull().sum()

병원명      0
주소       0
시도       0
시군구      0
시군구2     0
시도2      0
시도시군구    0
중분류      3
dtype: int64

결측치 확인 및 원인 파악

In [ ]:
# 결측치 3개 확인
temp_result[temp_result['중분류'].isnull()]

,병원명,주소,시도,시군구,시군구2,시도2,시도시군구,중분류
37,웰키즈소아청소년과의원,"세종특별자치시 보듬3로 95, 118,307,308,309-1,310-2,401~5...",세종특별자치시,보듬3로,"95,",세종특별자치시,세종특별자치시 보듬3로,NaN
85,서울패밀리병원,"경상남도 창원시 성산구 마디미서로 54, 서울아동병원(2,7,8,9,11,12층) ...",경상남도,창원시,성산구,경남,경남 창원시 성산구,NaN
86,양덕서울아동병원,경상남도 창원시 마산회원구 양덕로 190 (석전동),경상남도,창원시,마산회원구,경남,경남 창원시 마산회원구,NaN


In [ ]:
# 원인 분석
dong_temp_res['시도시군구'].unique()

# 1번
# 웰키즈소아청소년과의원
# dong_temp_res['시도시군구'] >> '세종특별자치시 세종특별자치시'
# moon_temp['시도시군구'] >> '세종특별자치시 보듬3로'
# 불일치

# 2번
# 서울패밀리병원
# dong_temp_res['시도시군구'] >> '경남 성산구'
# moon_temp['시도시군구'] >> '경남 창원시 성산구'
# 불일치

# 3번
# 양덕서울아동병원
# dong_temp_res['시도시군구'] >> '경남 마산회원구'
# moon_temp['시도시군구'] >> '경남 창원시 마산회원구'
# 불일치

# 창원시 관련 시도시군구 데이터 값 : ['경남 창원시', '경남 의창구', '경남 성산구', '경남 마산합포구', '경남 마산회원구']

# 원인 중분류(시군구코드)를 가진 데이터 파일이 3개가 존재
# 1. 우리 변수 데이터
# 2. 한국행정분류 데이터
# 3. KOSIS 데이터들
# 이 3가지의 분류 기준이 조금씩 달라서 계속 같은 오류가 발생함 ^^

array(['서울 종로구', '서울 중구', '서울 용산구', '서울 성동구', '서울 광진구', '서울 동대문구',
       '서울 중랑구', '서울 성북구', '서울 강북구', '서울 도봉구', '서울 노원구', '서울 은평구',
       '서울 서대문구', '서울 마포구', '서울 양천구', '서울 강서구', '서울 구로구', '서울 금천구',
       '서울 영등포구', '서울 동작구', '서울 관악구', '서울 서초구', '서울 강남구', '서울 송파구',
       '서울 강동구', '부산 중구', '부산 서구', '부산 동구', '부산 영도구', '부산 부산진구', '부산 동래구',
       '부산 남구', '부산 북구', '부산 해운대구', '부산 사하구', '부산 금정구', '부산 강서구',
       '부산 연제구', '부산 수영구', '부산 사상구', '부산 기장군', '대구 중구', '대구 동구', '대구 서구',
       '대구 남구', '대구 북구', '대구 수성구', '대구 달서구', '대구 달성군', '인천 중구', '인천 동구',
       '인천 연수구', '인천 남동구', '인천 부평구', '인천 계양구', '인천 서구', '인천 미추홀구',
       '인천 강화군', '인천 옹진군', '광주 동구', '광주 서구', '광주 남구', '광주 북구', '광주 광산구',
       '대전 동구', '대전 중구', '대전 서구', '대전 유성구', '대전 대덕구', '울산 중구', '울산 남구',
       '울산 동구', '울산 북구', '울산 울주군', '세종특별자치시 세종특별자치시', '경기 수원시', '경기 장안구',
       '경기 권선구', '경기 팔달구', '경기 영통구', '경기 성남시', '경기 수정구', '경기 중원구',
       '경기 분당구', '경기 의정부시', '경기 안양시', '경기 만안구', '경기 동안구', '경기 부천시',
       '경기 광명시', '경기 

In [ ]:
temp_result[temp_result['중분류'].isnull()]

,병원명,주소,시도,시군구,시군구2,시도2,시도시군구,중분류
37,웰키즈소아청소년과의원,"세종특별자치시 보듬3로 95, 118,307,308,309-1,310-2,401~5...",세종특별자치시,보듬3로,"95,",세종특별자치시,세종특별자치시 보듬3로,NaN
85,서울패밀리병원,"경상남도 창원시 성산구 마디미서로 54, 서울아동병원(2,7,8,9,11,12층) ...",경상남도,창원시,성산구,경남,경남 창원시 성산구,NaN
86,양덕서울아동병원,경상남도 창원시 마산회원구 양덕로 190 (석전동),경상남도,창원시,마산회원구,경남,경남 창원시 마산회원구,NaN


결측치에 넣어줄 값 확인

In [ ]:
dong_temp_res[dong_temp_res['시도시군구'] =='세종특별자치시 세종특별자치시']

,중분류,시도시군구
74,29010,세종특별자치시 세종특별자치시


In [ ]:
dong_temp_res[dong_temp_res['시도시군구'] =='경남 성산구']

,중분류,시도시군구
245,38112,경남 성산구


In [ ]:
dong_temp_res[dong_temp_res['시도시군구'] =='경남 마산회원구']

,중분류,시도시군구
247,38114,경남 마산회원구


In [ ]:
# '세종특별자치시 세종특별자치시' >> '29010'
# '경남 성산구' >> '38112'
# '경남 마산회원구' >> 38114

In [ ]:
# 결측치 수작업 처리
temp_result.loc[temp_result['병원명'] == '웰키즈소아청소년과의원', '중분류'] = '29010'

In [ ]:
temp_result.loc[temp_result['병원명'] == '웰키즈소아청소년과의원']

,병원명,주소,시도,시군구,시군구2,시도2,시도시군구,중분류
37,웰키즈소아청소년과의원,"세종특별자치시 보듬3로 95, 118,307,308,309-1,310-2,401~5...",세종특별자치시,보듬3로,"95,",세종특별자치시,세종특별자치시 보듬3로,29010


In [ ]:
temp_result.loc[temp_result['병원명'] == '서울패밀리병원', '중분류'] = '38112'
temp_result.loc[temp_result['병원명'] == '양덕서울아동병원', '중분류'] = '38114'

In [ ]:
temp_result.loc[temp_result['병원명'] == '서울패밀리병원']

,병원명,주소,시도,시군구,시군구2,시도2,시도시군구,중분류
85,서울패밀리병원,"경상남도 창원시 성산구 마디미서로 54, 서울아동병원(2,7,8,9,11,12층) ...",경상남도,창원시,성산구,경남,경남 창원시 성산구,38112


In [ ]:
temp_result.loc[temp_result['병원명'] == '양덕서울아동병원']

,병원명,주소,시도,시군구,시군구2,시도2,시도시군구,중분류
86,양덕서울아동병원,경상남도 창원시 마산회원구 양덕로 190 (석전동),경상남도,창원시,마산회원구,경남,경남 창원시 마산회원구,38114


In [ ]:
# 결측치 대체 완료
temp_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   병원명     90 non-null     object
 1   주소      90 non-null     object
 2   시도      90 non-null     object
 3   시군구     90 non-null     object
 4   시군구2    90 non-null     object
 5   시도2     90 non-null     object
 6   시도시군구   90 non-null     object
 7   중분류     90 non-null     object
dtypes: object(8)
memory usage: 5.8+ KB


결과물 저장 형태 맞추기
- 필요없는 열 삭제 및 열 재배치

In [ ]:
temp_result[:3]

,병원명,주소,시도,시군구,시군구2,시도2,시도시군구,중분류
0,의료법인우리아이들의료재단우리아이들병원,"서울특별시 구로구 새말로 15, 2~9층 (구로동, 삼전솔하임)",서울특별시,구로구,새말로,서울,서울 구로구,11170
1,24시열린의원,"서울특별시 광진구 능동로 90, 3층 (자양동, 더클래식500)",서울특별시,광진구,능동로,서울,서울 광진구,11050
2,연세곰돌이소아청소년과의원,"서울특별시 서초구 방배로 226, 넥센강남타워 3층 (방배동)",서울특별시,서초구,방배로,서울,서울 서초구,11220


In [ ]:
temp_result = temp_result.drop(['시도', '시군구', '시군구2', '시도2'], axis=1)

In [ ]:
temp_result = temp_result.set_index('중분류').reset_index()
temp_result

,중분류,병원명,주소,시도시군구
0,11170,의료법인우리아이들의료재단우리아이들병원,"서울특별시 구로구 새말로 15, 2~9층 (구로동, 삼전솔하임)",서울 구로구
1,11050,24시열린의원,"서울특별시 광진구 능동로 90, 3층 (자양동, 더클래식500)",서울 광진구
2,11220,연세곰돌이소아청소년과의원,"서울특별시 서초구 방배로 226, 넥센강남타워 3층 (방배동)",서울 서초구
3,11160,연세의원,"서울특별시 강서구 화곡로 197, 1,2층 (화곡동)",서울 강서구
4,11110,미즈아이프라자산부인과의원,"서울특별시 노원구 동일로 1669, 수락프라자 (상계동)",서울 노원구
...,...,...,...,...
85,38112,서울패밀리병원,"경상남도 창원시 성산구 마디미서로 54, 서울아동병원(2,7,8,9,11,12층) ...",경남 창원시 성산구
86,38114,양덕서울아동병원,경상남도 창원시 마산회원구 양덕로 190 (석전동),경남 창원시 마산회원구
87,39020,우리들소아과의원,"제주특별자치도 서귀포시 중정로 70, (서귀동)",제주 서귀포시
88,39010,탑동365일의원,"제주특별자치도 제주시 탑동로 24 (삼도이동,2층)",제주 제주시


In [ ]:
# temp_result.to_csv('./시군구코드별_병원정보(이름_주소).csv')

# 변수 모음 테이블에 병원 개수 오류 수정하기

In [ ]:
feature_df = df.copy()
feature_df

,중분류,행정구역시도명,행정구역시군구명,병원개수,인구 천명당 병원수,면적,소아청소년수,출생아수,가임연령부부수,2차병원수,3차병원수
0,31050,경기도,부천시,1.0,0.000555,53.45,2227,791,1203,8.0,0.0
1,31240,경기도,화성시,2.0,0.001120,698.18,2319,991,1450,10.0,0.0
2,31023,경기도,분당구,0.0,0.000000,69.49,2093,580,969,5.0,1.0
3,31130,경기도,남양주시,1.0,0.000679,458.14,1816,646,827,5.0,0.0
4,11230,서울특별시,강남구,2.0,0.001423,39.50,1634,369,576,7.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
245,32410,강원도,양양군,0.0,0.000000,630.01,99,42,53,0.0,0.0
246,37330,경상북도,청송군,0.0,0.000000,846.12,94,39,46,1.0,0.0
247,37340,경상북도,영양군,0.0,0.000000,815.75,78,24,24,0.0,0.0
248,37310,경상북도,군위군,0.0,0.000000,614.32,69,21,25,0.0,0.0


In [ ]:
col = ['중분류', '병원명']
hos_df = temp_result[col]
hos_df

,중분류,병원명
0,11170,의료법인우리아이들의료재단우리아이들병원
1,11050,24시열린의원
2,11220,연세곰돌이소아청소년과의원
3,11160,연세의원
4,11110,미즈아이프라자산부인과의원
...,...,...
85,38112,서울패밀리병원
86,38114,양덕서울아동병원
87,39020,우리들소아과의원
88,39010,탑동365일의원


In [ ]:
# 기준이 되는 열 중분류의 데이터 타입을 int로 맞춰준다
hos_df['중분류'] = hos_df['중분류'].astype(int)

<ipython-input-94-e3c79844f7b5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hos_df['중분류'] = hos_df['중분류'].astype(int)


In [ ]:
# 결과 확인
hos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   중분류     90 non-null     int64 
 1   병원명     90 non-null     object
dtypes: int64(1), object(1)
memory usage: 1.5+ KB


In [ ]:
hos_group = hos_df.groupby('중분류').count()
hos_group = hos_group.reset_index()
hos_group

,중분류,병원명
0,11050,1
1,11080,1
2,11110,1
3,11140,1
4,11150,1
...,...,...
64,38090,2
65,38112,1
66,38114,1
67,39010,2


In [ ]:
temp_res = feature_df.merge(hos_group, on='중분류', how='left', suffixes=('_feat','_hos'))

In [ ]:
# 결과 확인
temp_res['병원명'].sum()

# 오류해결 후 결과 재확인 결과 90개 잘 들어감

90.0

달빛 병원이 전국에 90개 있는데 왜 87개만 들어갔는가?
- 우리 데이터 250곳에 포함되지 않는 시군구코드를 가진 병원 3개 존재
- 해결완료

In [ ]:
# 오류해결완료로 인한 코드 주석화
# nope = []

# for i in range(len(hos_df)):
#     if hos_df['중분류'][i] in feature_df['중분류'].values:
#         pass
#     else:
#         nope.append(hos_df['중분류'][i])

In [ ]:
# nope

In [ ]:
# temp_result[temp_result['중분류'] == 31010]

In [ ]:
# temp_result[temp_result['중분류'] == 31020]

In [ ]:
# temp_result[temp_result['중분류'] == 31040]

In [ ]:
# map2

In [ ]:
temp_res

,중분류,행정구역시도명,행정구역시군구명,병원개수,인구 천명당 병원수,면적,소아청소년수,출생아수,가임연령부부수,2차병원수,3차병원수,병원명
0,31050,경기도,부천시,1.0,0.000555,53.45,2227,791,1203,8.0,0.0,1.0
1,31240,경기도,화성시,2.0,0.001120,698.18,2319,991,1450,10.0,0.0,2.0
2,31023,경기도,분당구,0.0,0.000000,69.49,2093,580,969,5.0,1.0,NaN
3,31130,경기도,남양주시,1.0,0.000679,458.14,1816,646,827,5.0,0.0,1.0
4,11230,서울특별시,강남구,2.0,0.001423,39.50,1634,369,576,7.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
245,32410,강원도,양양군,0.0,0.000000,630.01,99,42,53,0.0,0.0,NaN
246,37330,경상북도,청송군,0.0,0.000000,846.12,94,39,46,1.0,0.0,NaN
247,37340,경상북도,영양군,0.0,0.000000,815.75,78,24,24,0.0,0.0,NaN
248,37310,경상북도,군위군,0.0,0.000000,614.32,69,21,25,0.0,0.0,NaN


In [ ]:
result_fin = temp_res.copy()
result_fin

,중분류,행정구역시도명,행정구역시군구명,병원개수,인구 천명당 병원수,면적,소아청소년수,출생아수,가임연령부부수,2차병원수,3차병원수,병원명
0,31050,경기도,부천시,1.0,0.000555,53.45,2227,791,1203,8.0,0.0,1.0
1,31240,경기도,화성시,2.0,0.001120,698.18,2319,991,1450,10.0,0.0,2.0
2,31023,경기도,분당구,0.0,0.000000,69.49,2093,580,969,5.0,1.0,NaN
3,31130,경기도,남양주시,1.0,0.000679,458.14,1816,646,827,5.0,0.0,1.0
4,11230,서울특별시,강남구,2.0,0.001423,39.50,1634,369,576,7.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
245,32410,강원도,양양군,0.0,0.000000,630.01,99,42,53,0.0,0.0,NaN
246,37330,경상북도,청송군,0.0,0.000000,846.12,94,39,46,1.0,0.0,NaN
247,37340,경상북도,영양군,0.0,0.000000,815.75,78,24,24,0.0,0.0,NaN
248,37310,경상북도,군위군,0.0,0.000000,614.32,69,21,25,0.0,0.0,NaN


In [ ]:
result_fin = result_fin.fillna(0)
result_fin = result_fin.drop('병원개수', axis=1)
result_fin.columns = ['중분류', '행정구역시도명', '행정구역시군구명', '인구 천명당 병원수', '면적', '소아청소년수',
       '출생아수', '가임연령부부수', '2차병원수', '3차병원수', '달빛병원수']

,중분류,행정구역시도명,행정구역시군구명,인구 천명당 병원수,면적,소아청소년수,출생아수,가임연령부부수,2차병원수,3차병원수,달빛병원수
0,31050,경기도,부천시,0.000555,53.45,2227,791,1203,8.0,0.0,1.0
1,31240,경기도,화성시,0.001120,698.18,2319,991,1450,10.0,0.0,2.0
2,31023,경기도,분당구,0.000000,69.49,2093,580,969,5.0,1.0,0.0
3,31130,경기도,남양주시,0.000679,458.14,1816,646,827,5.0,0.0,1.0
4,11230,서울특별시,강남구,0.001423,39.50,1634,369,576,7.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
245,32410,강원도,양양군,0.000000,630.01,99,42,53,0.0,0.0,0.0
246,37330,경상북도,청송군,0.000000,846.12,94,39,46,1.0,0.0,0.0
247,37340,경상북도,영양군,0.000000,815.75,78,24,24,0.0,0.0,0.0
248,37310,경상북도,군위군,0.000000,614.32,69,21,25,0.0,0.0,0.0


In [ ]:
result_fin = result_fin.drop('인구 천명당 병원수', axis=1)

In [ ]:
result_fin

,중분류,행정구역시도명,행정구역시군구명,면적,소아청소년수,출생아수,가임연령부부수,2차병원수,3차병원수,달빛병원수
0,31050,경기도,부천시,53.45,2227,791,1203,8.0,0.0,1.0
1,31240,경기도,화성시,698.18,2319,991,1450,10.0,0.0,2.0
2,31023,경기도,분당구,69.49,2093,580,969,5.0,1.0,0.0
3,31130,경기도,남양주시,458.14,1816,646,827,5.0,0.0,1.0
4,11230,서울특별시,강남구,39.50,1634,369,576,7.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...
245,32410,강원도,양양군,630.01,99,42,53,0.0,0.0,0.0
246,37330,경상북도,청송군,846.12,94,39,46,1.0,0.0,0.0
247,37340,경상북도,영양군,815.75,78,24,24,0.0,0.0,0.0
248,37310,경상북도,군위군,614.32,69,21,25,0.0,0.0,0.0


In [ ]:
%pwd

'/content'

In [ ]:
# 최종 결과물 저장
result_fin.to_csv('/content/drive/MyDrive/mulcam_bigdata/2024통계데이터공모전/기획_데이터수집/성원님/temp/시군구코드별_변수_v3.csv', index=False)

In [ ]:
# 저장 결과 확인
path = '/content/drive/MyDrive/mulcam_bigdata/2024통계데이터공모전/기획_데이터수집/성원님/temp/시군구코드별_변수_v3.csv'
pd.read_csv(path)

,중분류,행정구역시도명,행정구역시군구명,면적,소아청소년수,출생아수,가임연령부부수,2차병원수,3차병원수,달빛병원수
0,31050,경기도,부천시,53.45,2227,791,1203,8.0,0.0,1.0
1,31240,경기도,화성시,698.18,2319,991,1450,10.0,0.0,2.0
2,31023,경기도,분당구,69.49,2093,580,969,5.0,1.0,0.0
3,31130,경기도,남양주시,458.14,1816,646,827,5.0,0.0,1.0
4,11230,서울특별시,강남구,39.50,1634,369,576,7.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...
245,32410,강원도,양양군,630.01,99,42,53,0.0,0.0,0.0
246,37330,경상북도,청송군,846.12,94,39,46,1.0,0.0,0.0
247,37340,경상북도,영양군,815.75,78,24,24,0.0,0.0,0.0
248,37310,경상북도,군위군,614.32,69,21,25,0.0,0.0,0.0
